# Data Wrangling
## Business Problem
Leukemia is a type of cancer of the blood that often affects young people. In the past, pathologists would diagnose patients by eye after examining blood smear images under the microscope. But, this is time consuming and tedious. Advances in image recognition technology have come a long ways since their inception. Therefore, automated solutions using computers would be of great benefit to the medical community to aid in cancer diagnoses.

The goal of this project is to address the following question: How can the doctor’s at the Munich University Hospital automate the diagnosis of patients with leukemia using images from blood smears?

**Plan:**
Address the following questions:
* How many different morphology classes are there in the dataset?
* What are the image counts per morphology class?
* For each morphology class, what percentage of samples were re-annotated?
* Are there any samples there were re-annoted only once?

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Load Annotations File
The above questions can be answered by exmaining the annotations file, _annotations.dat_.